In [14]:
import numpy as np
import scipy
import pandas as pd

In [20]:
vocab = pd.read_csv('vocab.nips.txt', header=None)

D = 1500
W = 12419

doc_word = np.zeros((D, W))

doc = open('docword.nips.txt')
lines = doc.readlines()
for line in lines[3:]:
    split = line.split(" ")
    docID = int(split[0])
    wordID = int(split[1])
    count = int(split[2])
    doc_word[docID-1][wordID-1] = count